In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

train.info()
train.head(5)

In [ ]:
y = train['label']
x = train.drop(columns = ['label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
plt.figure(figsize=(12,8))

x, y = 10, 4

for i in range(20):  
    plt.subplot(y, x, i+1)
    plt.imshow(np.array(x_train.iloc[i]).reshape(28, 28), interpolation='nearest', cmap = plt.cm.binary)

plt.show()

In [ ]:
# Scale the data so that the values are from 0 - 1
x_train = x_train / 255
x_test = x_test / 255
test = test / 255


# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

x_train.shape, x_test.shape

In [ ]:
# Data Augmentation

datagen = ImageDataGenerator(
                            featurewise_center=False,  # set input mean to 0 over the dataset
                            samplewise_center=False,  # set each sample mean to 0
                            featurewise_std_normalization=False,  # divide inputs by std of the dataset
                            samplewise_std_normalization=False,  # divide each input by its std
                            zca_whitening=False,  # dimesion reduction
                            rotation_range=5,  # randomly rotate images in the range 5 degrees
                            zoom_range = 0.1, # Randomly zoom image 10%
                            width_shift_range=0.1,  # randomly shift images horizontally 10%
                            height_shift_range=0.1,  # randomly shift images vertically 10%
                            horizontal_flip=False,  # randomly flip images
                            vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
# Flattening data using keras Flatten class
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', input_shape = (28,28,1)))
model.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', input_shape = (28,28,1)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation ='relu', input_shape = (28,28,1)))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Flatten())                       
model.add(keras.layers.Dense(128, activation = 'relu')) # basic layer
model.add(keras.layers.Dense(10, activation = 'softmax')) # number of outputs expected, i.e 10 digits

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# model.fit(x_train, y_train, epochs = 5)

model.fit(datagen.flow(x_train, y_train), epochs = 5, validation_data = (x_test, y_test), verbose = 1)

In [ ]:
model.summary()

In [ ]:
y_predicted = model.predict(x_test)
y_predicted_labels = [np.argmax(i) for i in y_predicted]

cm = tf.math.confusion_matrix(labels=y_test, predictions=y_predicted_labels)
print("Accuracy score:", round(accuracy_score(y_test, y_predicted_labels), 3))

plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap = 'YlGnBu_r')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
digit = 42

y_predicted = model.predict(x_test)
print("Predicted number:", np.argmax(y_predicted[digit]))

print("\nReal number:", y_test.iloc[digit])
plt.imshow(x_test[digit], interpolation='nearest', cmap = plt.cm.binary)
plt.show()

In [ ]:
digit = 5

y_predicted = model.predict(x_test)
print("Predicted number:", np.argmax(y_predicted[digit]))

print("\nReal number:", y_test.iloc[digit])
plt.imshow(x_test[digit], interpolation='nearest', cmap = plt.cm.binary)
plt.show()

In [ ]:
submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

predictions = np.argmax(model.predict(test), axis = 1)
submission["Label"] = predictions
submission.to_csv("submission.csv",index = False)
submission